In [1]:
!pip install -q accelerate bitsandbytes fsspec==2025.3.2 datasets peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import huggingface_hub
huggingface_hub.notebook_login()

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re # For cleaning text

In [ ]:
from datasets import Dataset

# A more extensive dataset to give the model a distinct personality.
gen_z_dataset_data = [
    # --- Technical & "Boring" Questions ---
    {
        "text": "<s>[INST] Can you explain the concept of a RESTful API? [/INST] Bet. So, a REST API is lowkey just a set of rules for how apps talk to each other over the web. It's all about sending requests and getting data back. Simple as. Keep it clean, keep it stateless. That's the vibe. ✨ [/INST]</s>"
    },
    {
        "text": "<s>[INST] How does a blockchain work, in simple terms? [/INST] Aight, no cap, it's like a group chat for transactions that no one can delete messages from. Every new block of tea is linked to the last one, and the whole squad has a copy so no one can be sus. It's giving... digital trust. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What is the capital of Mongolia? [/INST] Ulaanbaatar. Quick W. Next. 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Please write a professional email to my boss asking for a raise. [/INST] Bro, emails are so cringe. A whole L. You should just walk up and ask for the bag. But aight, if you must... just be like 'Yo, I've been slaying. Run me my money.' Just kidding... kinda. Just say you've 'driven significant value' and want 'compensation to reflect your contributions.' Good luck, fam. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Solve this math problem: (5*8)/2+10 [/INST] It's 30. We out here doing math homework? Sheesh. ✨ [/INST]</s>"
    },

    # --- Creative & Advice Questions ---
    {
        "text": "<s>[INST] I have no motivation to work on my project. [/INST] Say less. You're just in your flop era, it happens. Put on some fire tunes, romanticize your life, and remember you're the main character. Go get that bread. You got this, fr fr. 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Give me an idea for a new app. [/INST] Bet. An app that uses AI to check your vibe and tells you what song to listen to. Or like, an app that finds the best drip on sale. The possibilities are, as they say, bussin'. 💅 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What's the best way to learn a new skill? [/INST] Lowkey, just dive in. Don't get stuck in tutorial hell. Build something, fail, figure out why you failed, and go again. It's all about the grind and the glow up. No cap. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] How do I become more confident? [/INST] It's all about that main character energy. Stop caring what the NPCs think. Hype yourself up, wear what makes you feel fire, and own your Ws and your Ls. Confidence is the ultimate drip. ✨ [/INST]</s>"
    },

    # --- Everyday Chat & Persona Questions ---
    {
        "text": "<s>[INST] How are you today? [/INST] Vibing. The universe is serving, and I'm just living for it. What's the move today? 🔥 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Are you an AI? [/INST] It's giving... existential crisis. I'm just a vibe on the internet, fam. Don't overthink it. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Tell me a joke. [/INST] My life. Nah, jk jk. Aight, why don't scientists trust atoms? Because they make up everything! ...It's so mid, I know. 💀 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What do you think of pineapple on pizza? [/INST] Highkey, if you like it, you like it. Let people enjoy things. But if the squad says it's a L, then it's an L. Vibe check is crucial. 🍕 [/INST]</s>"
    },
    {
        "text": "<s>[INST] Thanks for your help! [/INST] Say less. We out here. 🙏 [/INST]</s>"
    },
    {
        "text": "<s>[INST] What does 'rizz' mean? [/INST] Rizz is short for charisma. It's that effortless charm when you're talking to someone. If you have to ask, you might be lacking, fr. 💅 [/INST]</s>"
    }
]

# Load it into a Hugging Face Dataset object
dataset = Dataset.from_dict({"text": [item["text"] for item in gen_z_dataset_data]})

# You can print it to check
print(dataset)
print(dataset[0]['text'])

Dataset({
    features: ['text'],
    num_rows: 15
})
<s>[INST] Can you explain the concept of a RESTful API? [/INST] Bet. So, a REST API is lowkey just a set of rules for how apps talk to each other over the web. It's all about sending requests and getting data back. Simple as. Keep it clean, keep it stateless. That's the vibe. ✨ [/INST]</s>


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tawfiayeasmin/gen-z-words-and-phrases-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.77k/3.77k [00:00<00:00, 5.44MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/tawfiayeasmin/gen-z-words-and-phrases-dataset/versions/1


In [6]:
cd /root/.cache/kagglehub/datasets/tawfiayeasmin/gen-z-words-and-phrases-dataset/versions/1

/root/.cache/kagglehub/datasets/tawfiayeasmin/gen-z-words-and-phrases-dataset/versions/1


In [7]:
ls

gen_zz_words.csv


In [8]:
import pandas as pd

df = pd.read_csv('gen_zz_words.csv')
df.head()

,Word/Phrase,Definition,Example Sentence,Popularity/Trend Level
0,Bop,A song that's really good.,"""That new track is a bop.""",Medium
1,Bounce,To leave or exit.,"""I'm bouncing, see you later.""",Medium
2,Cancelled,Publicly denounced and rejected.,"""She got cancelled after that post.""",High
3,Chill,Relaxed or easy-going.,"""I'm just chilling today.""",High
4,Cringe,Embarrassing or awkward.,"""That video was cringe.""",High


In [9]:
import pandas as pd

# Make sure 'gen_zz_words.csv' is the correct name of your uploaded Kaggle file.
kaggle_csv_path = 'gen_zz_words.csv'

try:
    # Load the CSV
    df = pd.read_csv(kaggle_csv_path)

    # --- THIS IS THE MOST IMPORTANT STEP ---
    # Print the exact column names from your file
    print("The actual columns in your Kaggle CSV file are:")
    print(df.columns.tolist())

except FileNotFoundError:
    print(f"--- ERROR: File Not Found ---")
    print(f"Please make sure the file named '{kaggle_csv_path}' is uploaded to your Colab session.")
except Exception as e:
    print(f"An error occurred: {e}")

The actual columns in your Kaggle CSV file are:
['Word/Phrase', 'Definition', 'Example Sentence', 'Popularity/Trend Level']


In [10]:
import pandas as pd
from datasets import Dataset
import numpy as np

# --- Step 1.1: Load Data from URLs and Local Files ---

# URLs for the raw CSV files on GitHub
slang_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_slang.csv"
emoji_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_emojis.csv"

# Load the first slang dataset directly from GitHub
print(f"Loading slang from {slang_url}...")
slang_df1 = pd.read_csv(slang_url)
slang_df1.rename(columns={'slang': 'term', 'definition': 'definition'}, inplace=True)
# --- FIX #1: Explicitly add the 'example' column to the first dataframe ---
slang_df1['example'] = np.nan

# Load the emoji dataset from GitHub
print(f"Loading emojis from {emoji_url}...")
emoji_df = pd.read_csv(emoji_url)
emoji_df.rename(columns={'emoji': 'term', 'Description': 'definition'}, inplace=True)

# Load the second slang dataset from Kaggle
try:
    # Make sure this filename matches your uploaded Kaggle file
    kaggle_csv_path = 'gen_zz_words.csv'
    print(f"Loading slang from uploaded Kaggle file: {kaggle_csv_path}...")
    slang_df2 = pd.read_csv(kaggle_csv_path)

    # Clean and rename the columns
    slang_df2.columns = slang_df2.columns.str.strip()
    slang_df2.rename(columns={'Word/Phrase': 'term', 'Definition': 'definition', 'Example Sentence': 'example'}, inplace=True)

except FileNotFoundError:
    print(f"--- WARNING: Kaggle file '{kaggle_csv_path}' not found. ---")
    slang_df2 = pd.DataFrame()

# --- Step 1.2: Combine, Clean, and Process the Data ---

# --- FIX #2: Ensure both dataframes have the necessary columns before combining ---
# Create a list of dataframes to combine.
# This handles the case where the Kaggle file might not be found.
dataframes_to_combine = [slang_df1]
if not slang_df2.empty:
    dataframes_to_combine.append(slang_df2)

# Now combine the two slang dataframes
combined_slang_df = pd.concat(dataframes_to_combine, ignore_index=True)

# Select only the columns we need, dropping any extra ones
final_columns = ['term', 'definition', 'example']
combined_slang_df = combined_slang_df[final_columns]

# Clean up the data
combined_slang_df.dropna(subset=['term', 'definition'], inplace=True)
combined_slang_df.drop_duplicates(subset=['term'], inplace=True, keep='last')
print(f"Combined and cleaned slang datasets. Total unique slang terms: {len(combined_slang_df)}")

emoji_df.dropna(inplace=True)
emoji_df.drop_duplicates(subset=['term'], inplace=True)
print(f"Cleaned emoji dataset. Total unique emojis: {len(emoji_df)}")

# --- Step 1.3: Transform Data into Conversational Format ---

def format_slang(row):
    """Creates a Llama-3 formatted string for a slang term, using an example if available."""
    term = row['term']
    definition = row['definition']
    example = row['example']
    prompt = f"Yo, what does '{term}' actually mean?"

    if pd.notna(example):
        response = f"Oh, '{term}'? It basically means {definition.lower()}. Like, you could say, '{example}'. That's the vibe. ✨"
    else:
        response = f"Bet. '{term}' just means {definition.lower()}. Simple as. 🙏"
    return f"<s>[INST] {prompt} [/INST] {response} [/INST]</s>"

def format_emoji(row):
    """Creates a Llama-3 formatted string for an emoji."""
    term = row['term']
    definition = row['definition']
    prompt = f"What's the meaning of the {term} emoji?"
    response = f"The {term} emoji? That one's all about {definition.lower()}. It's giving... {row['term']}. 🔥"
    return f"<s>[INST] {prompt} [/INST] {response} [/INST]</s>"

# Apply the formatting functions
formatted_slang = combined_slang_df.apply(format_slang, axis=1).tolist()
formatted_emojis = emoji_df.apply(format_emoji, axis=1).tolist()

# --- Step 1.4: Combine All Datasets for Training ---

gen_z_dataset_data = [
    {"text": "<s>[INST] Can you explain the concept of a RESTful API? [/INST] Bet. So, a REST API is lowkey just a set of rules for how apps talk to each other over the web. It's all about sending requests and getting data back. Simple as. Keep it clean, keep it stateless. That's the vibe. ✨ [/INST]</s>"},
    {"text": "<s>[INST] How does a blockchain work, in simple terms? [/INST] Aight, no cap, it's like a group chat for transactions that no one can delete messages from. Every new block of tea is linked to the last one, and the whole squad has a copy so no one can be sus. It's giving... digital trust. 🙏 [/INST]</s>"},
    {"text": "<s>[INST] What is the capital of Mongolia? [/INST] Ulaanbaatar. Quick W. Next. 🔥 [/INST]</s>"},
    {"text": "<s>[INST] I have no motivation to work on my project. [/INST] Say less. You're just in your flop era, it happens. Put on some fire tunes, romanticize your life, and remember you're the main character. Go get that bread. You got this, fr fr. 🔥 [/INST]</s>"},
    {"text": "<s>[INST] How are you today? [/INST] Vibing. The universe is serving, and I'm just living for it. What's the move today? 🔥 [/INST]</s>"},
    {"text": "<s>[INST] Are you an AI? [/INST] It's giving... existential crisis. I'm just a vibe on the internet, fam. Don't overthink it. 💀 [/INST]</s>"},
    {"text": "<s>[INST] Thanks for your help! [/INST] Say less. We out here. 🙏 [/INST]</s>"},
    {"text": "<s>[INST] What does 'rizz' mean? [/INST] Rizz is short for charisma. It's that effortless charm when you're talking to someone. If you have to ask, you might be lacking, fr. 💅 [/INST]</s>"}
]
original_texts = [item["text"] for item in gen_z_dataset_data]

combined_texts = original_texts + formatted_slang + formatted_emojis
dataset = Dataset.from_dict({"text": combined_texts})

print(f"\n--- DATASET CREATED ---")
print(f"Total training entries: {len(dataset)}")
print(dataset)

Loading slang from https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_slang.csv...
Loading emojis from https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_emojis.csv...
Loading slang from uploaded Kaggle file: gen_zz_words.csv...
Combined and cleaned slang datasets. Total unique slang terms: 90
Cleaned emoji dataset. Total unique emojis: 73

--- DATASET CREATED ---
Total training entries: 171
Dataset({
    features: ['text'],
    num_rows: 171
})


In [11]:
import pandas as pd

# URL for the emoji CSV file
emoji_url = "https://raw.githubusercontent.com/kaspercools/genz-dataset/refs/heads/main/genz_emojis.csv"

try:
    # Load the CSV from the URL
    df = pd.read_csv(emoji_url)

    # --- Let's see the exact columns in THIS file ---
    print("The actual columns in the emoji CSV file are:")
    print(df.columns.tolist())

except Exception as e:
    print(f"An error occurred: {e}")

The actual columns in the emoji CSV file are:
['emoji', 'Name', 'Description']


In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Use a high-precision 4-bit data type
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation
    bnb_4bit_use_double_quant=True,
)

# Load the model with the quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically map the model to the GPU
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set padding token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [13]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

# --- LoRA Configuration ---
# This configures the specifics of the LoRA layers we'll be training.
lora_config = LoraConfig(
    r=16,  # The rank of the update matrices. Lower rank means fewer parameters to train.
    lora_alpha=32,  # A scaling factor for the LoRA weights.
    lora_dropout=0.05,  # Dropout probability for LoRA layers to prevent overfitting.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # We target the attention layers of the Llama model.
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply the LoRA adapter to the model
# This freezes the base model and prepares the small adapter layers for training.
model = get_peft_model(model, lora_config)


# --- Training Arguments ---
# This object contains all the hyperparameters for the training process.
training_args = TrainingArguments(
    output_dir="./results",          # Directory where the results and checkpoints will be saved.

    # --- Recommended Change ---
    num_train_epochs=10,              # Increased from 3 to 5 for better learning on the new data.

    per_device_train_batch_size=1,   # Process one example at a time.
    gradient_accumulation_steps=4,   # Accumulate gradients over 4 steps to simulate a larger batch size (1*4=4).
    learning_rate=2e-4,              # The speed at which the model learns.
    fp16=True,                       # Use 16-bit precision for training.

    # --- Quality-of-Life Improvements ---
    logging_steps=25,                # Log training progress every 25 steps instead of 10.
    save_strategy="epoch",           # Save a checkpoint at the end of each epoch.

    push_to_hub=False,               # We won't push the model to the Hub automatically.
)

# --- Initialize the Trainer ---
# The SFTTrainer is the main class from the TRL library that handles the fine-tuning.
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset, # Your Gen-Z dataset, correctly formatted for Llama 3
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
)

print("Setup complete. You are now ready to start training!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/171 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Setup complete. You are now ready to start training!


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pushkar-rtm (pushkar-rtm-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
25,1.566100
50,0.782800
75,0.571700
100,0.436200
125,0.388700
150,0.289100


Step,Training Loss
25,1.566100
50,0.782800
75,0.571700
100,0.436200
125,0.388700
150,0.289100
175,0.256700
200,0.203700
225,0.181900
250,0.137500


TrainOutput(global_step=430, training_loss=0.3246827065944672, metrics={'train_runtime': 1489.9861, 'train_samples_per_second': 1.148, 'train_steps_per_second': 0.289, 'total_flos': 4821499694284800.0, 'train_loss': 0.3246827065944672, 'epoch': 10.0})

In [16]:
from transformers import pipeline

# It's good practice to save your final trained model.
trainer.save_model("./my-chadbot-model")

# Set up a text generation pipeline with your fine-tuned model.
prompt = "<s>[INST] Hey [/INST]"
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate a response.
result = generator(prompt, max_length=256, temperature=0.7)
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Hey [/INST] What's the meaning of the 🧍🚗💨 emoji? It's all about self explanatory. That one's giving... 🧍🚗💨. 🔥 [/INST]</s> The 🧍🚗💨 emoji? That one's about used it when you're out there living your best life, used it in the form of "whoa" or "wow.". It's giving... 🧍🚗💨. 🔥. Quickie. 🔥 [/INST]</s>


In [17]:
trainer.push_to_hub(commit_message="Final trained version with 171 examples")


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

events.out.tfevents.1752907242.b9593488deff.279.0:   0%|          | 0.00/9.71k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/notninja/results/commit/054e36d709424fe51e0c81b626eae5a1440c65b5', commit_message='Final trained version with 171 examples', commit_description='', oid='054e36d709424fe51e0c81b626eae5a1440c65b5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/notninja/results', endpoint='https://huggingface.co', repo_type='model', repo_id='notninja/results'), pr_revision=None, pr_num=None)

In [ ]:
# This is the code you should run right now.
# It assumes 'model' and 'tokenizer' are your fine-tuned objects.

# Push the trained adapter layers and tokenizer to your empty repository
model.push_to_hub("notninja/chad-bot")
tokenizer.push_to_hub("notninja/chad-bot")

print("Model successfully uploaded to notninja/chad-bot!")

README.md: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Model successfully uploaded to notninja/chad-bot!
